In [1]:
from src.dl.loader import BlockClassifyDataset
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, sampler
from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import KFold
import copy

In [2]:
writer = SummaryWriter('./log')

In [1]:
from src.dl.loader import BlockClassifyDataset
from src.dl.resnet import ClassifyResNet3d
from sklearn.model_selection import KFold
import collections
bounds = [0.2, 0.5]
train_set = BlockClassifyDataset('test', bounds)
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)

TypeError: __init__() missing 1 required positional argument: 'fold_num'

In [5]:
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(train_set)))):
    net = ClassifyResNet3d(len(bounds)+1).cuda()
    loss_fn = nn.CrossEntropyLoss()
    lr = 1e-2
    optim = torch.optim.Adam(net.parameters(), lr=lr, betas=[0.9, 0.99])
    epoch = 40
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(train_set, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(train_set, batch_size=64, sampler=val_sampler)
    best_acc = 0
    for i in range(epoch):
        total_loss = 0
        predy = np.array([])
        y = np.array([])
        net.train()
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels)
            loss = loss_fn(output, score)

            optim.zero_grad()
            loss.backward()
            optim.step()
            #scheduler.step()
        
        net.eval()
        with torch.no_grad():
            total = 0
            correct = 0
            for step, [img, labels, score] in enumerate(val_loader):
                img = img.cuda()
                labels = labels.cuda()
                score = score.cuda()
                output = net(img, labels)
                loss = loss_fn(output, score)
                
                _, predicted = torch.max(output.data, 1)
                total += score.size(0)
                correct += np.sum(predicted.cpu().detach().numpy() == score.cpu().detach().numpy())
                total_loss += loss.item()

        acc = 100 * correct / total
        if acc >= best_acc:
            best_acc = acc
            best_models[fold] = copy.deepcopy(net.state_dict())
        print('fold {}, epoch {}, loss {}, acc {}%'.format(fold, i, total_loss, acc))
        #writer.add_scalar('Loss/loss'+str(fold), total_loss, i)
        #writer.add_scalar('ACC/acc'+str(fold), acc, i)
    best_acc = 0
    break

#writer.close()

fold 0, epoch 0, loss 25.36275154352188, acc 52.2289156626506%
fold 0, epoch 1, loss 25.79340159893036, acc 53.01204819277108%
fold 0, epoch 2, loss 66.44929361343384, acc 45.903614457831324%
fold 0, epoch 3, loss 8.27451991289854, acc 88.55421686746988%
fold 0, epoch 4, loss 16.002849817276, acc 80.60240963855422%
fold 0, epoch 5, loss 2.3077152324840426, acc 95.12048192771084%
fold 0, epoch 6, loss 4.3462723679840565, acc 93.07228915662651%
fold 0, epoch 7, loss 3.69453813880682, acc 93.01204819277109%
fold 0, epoch 8, loss 136.72240436077118, acc 76.56626506024097%


KeyboardInterrupt: 

In [6]:
test_set = BlockClassifyDataset('train', bounds)
test_loader = DataLoader(test_set, batch_size=64)
net.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for step, [img, labels, score] in enumerate(test_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels)
        loss = loss_fn(output, score)
        
        _, predicted = torch.max(output.data, 1)
        total += score.size(0)
        correct += np.sum(predicted.cpu().detach().numpy() == score.cpu().detach().numpy())
        total_loss += loss.item()
acc = 100 * correct / total
print('loss {}, acc {}%'.format(total_loss, acc))

loss 3659.054661643261, acc 42.23132969034609%
